In [4]:
import numpy as np
from scipy.optimize import minimize
import autograd.numpy as anp
from autograd import grad, hessian
import pandas as pd
import time
import copy

def goldstein_price(x):
    term1 = (1 + (x[0] + x[1] + 1)**2 * (19 - 14*x[0] + 3*x[0]**2 - 14*x[1] + 6*x[0]*x[1] + 3*x[1]**2))
    term2 = (30 + (2*x[0] - 3*x[1])**2 * (18 - 32*x[0] + 12*x[0]**2 + 48*x[1] - 36*x[0]*x[1] + 27*x[1]**2))
    return term1 * term2

gradient = grad(goldstein_price)
hessian_matrix = hessian(goldstein_price)

def line_search_function(t, x, direction):
    return goldstein_price(x + t * direction)

def minimize_along_direction(x, direction):
    result = minimize(line_search_function, 0, args=(x, direction), method='BFGS')
    t_optimal = result.x[0]
    if t_optimal < 0:
        t_optimal = 0  # Si t es negativo, establecerlo en 0
    return t_optimal

tol=1e-5
i_max=10000

puntos= [np.random.uniform(-2, 2, size=2) for _ in range(10)]
df_puntos=pd.DataFrame(puntos)
puntos_random=copy.deepcopy(puntos)

#Metodo Gradiente.
Resultados_Gradiente = pd.DataFrame(columns=("x1*","x2*","i","f","t"))
for x_r in puntos:
    i=0
    ini=time.time()
    while True:
        dir = -gradient(x_r)  
        t_opt = minimize_along_direction(x_r, dir)
        parada=np.linalg.norm(gradient(x_r))
        if parada<tol:
            break
        else:
            x_r[0]=x_r[0]+t_opt*dir[0]
            x_r[1]=x_r[1]+t_opt*dir[1]
        i+=1
        if i>i_max:
            break
    fin=time.time()
    tiempo=fin-ini
    Resultados_Gradiente.loc[len(Resultados_Gradiente.index)]=[x_r[0],x_r[1],i,goldstein_price(x_r),tiempo]
print(Resultados_Gradiente)


#Metodo Newton.
Resultados_Newton = pd.DataFrame(columns=("x1*","x2*","i","f","t"))
for x_r in puntos_random:
    i=0
    ini=time.time()
    while True:
        hessian_inv_x_r = np.linalg.inv(hessian_matrix(x_r))
        dir = -np.dot(hessian_inv_x_r, gradient(x_r))
        t_opt = minimize_along_direction(x_r, dir)
        parada = np.dot(gradient(x_r).T, np.dot(hessian_inv_x_r, gradient(x_r))) / 2
        if parada<tol:
            break
        else:
            x_r[0]=x_r[0]+t_opt*dir[0]
            x_r[1]=x_r[1]+t_opt*dir[1]
        i=i+1
        if i>i_max:
            break
    fin=time.time()
    tiempo=fin-ini
    Resultados_Newton.loc[len(Resultados_Newton.index)]=[x_r[0],x_r[1],i,goldstein_price(x_r),tiempo]
print(Resultados_Newton)

        x1*       x2*        i      f          t
0  0.000001 -0.999999  10001.0    3.0  24.313133
1 -0.600004 -0.399996  10001.0   30.0  24.125219
2  1.200001  0.800001  10001.0  840.0  24.395226
3 -0.000005 -1.000003  10001.0    3.0  25.711615
4  0.000001 -1.000003  10001.0    3.0  24.097908
5 -0.600005 -0.399996  10001.0   30.0  24.186826
6  0.000007 -0.999999  10001.0    3.0  24.157759
7  1.800011  0.200007  10001.0   84.0  89.998962
8  0.000006 -0.999999  10001.0    3.0  27.202239
9  0.000005 -0.999998  10001.0    3.0  24.238434
            x1*       x2*        i           f           t
0 -5.530019e-07 -1.000001      2.0    3.000000    0.028398
1  2.591397e-05 -1.000022      3.0    3.000001    0.036238
2  1.199964e+00  0.800038  10001.0  840.000018  170.775972
3 -8.294258e-06 -0.999992      2.0    3.000000    0.027665
4  8.398058e-02  0.029510      1.0  681.450698    0.021137
5 -3.350342e-02 -0.049561      1.0  541.097217    0.020186
6 -8.184407e-05 -1.000050      3.0    3.000002  